In [1]:
import os
import hydra
import hydra.experimental
import numpy as np
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
import tqdm
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaTokenizer
from omegaconf import DictConfig
from fast_bleu import BLEU, SelfBLEU

from torchfly.text.decode import TransformerDecoder
from torchfly.common import set_random_seed, move_to_device

from configure_dataloader import DataLoaderHandler

import logging
import dill
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 16})

In [2]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [3]:
def ref_data():
    data = []
    with open('../StyleCaption/funny-test.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower())
            line = f.readline()
    with open('../StyleCaption/fact-test.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower())
            line = f.readline()
    
    with open('../StyleCaption/romantic-test.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower())
            line = f.readline()
    return data

In [4]:
def gen_data():
    data = []
    with open('./funny-GEN.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower().replace("\"",""))
            line = f.readline()
    with open('./fact-GEN.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower().replace("\"",""))
            line = f.readline()
    
    with open('./romantic-GEN.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower().replace("\"",""))
            line = f.readline()
    return data

In [64]:
ref =ref_data()
gen = gen_data()

In [65]:
# print(gen)

In [66]:
# print(ref)

In [67]:
ref = [tokenizer.encode(item, add_special_tokens=False) for item in ref]
bleu = BLEU(ref, {'4gram': (0.25, 0.25, 0.25, 0.25)})

In [68]:
gen = [tokenizer.encode(item.lower(), add_special_tokens=False) for item in gen]

In [69]:
bleu_score = np.mean(bleu.get_score(gen)['4gram'])
self_bleu = SelfBLEU(gen, {'4gram': (0.25,0.25,0.25,0.25)})
self_bleu_score = np.mean(self_bleu.get_score()['4gram'])

In [70]:
print(bleu_score)

0.5407276724324203


In [71]:
print(self_bleu_score)

0.8208848928911244
